# Basic Model Interface (BMI) for streamflow routing using t-route

This t-route network was developed for use in the Next Generation National Water Model (Nextgen). Nextgen runs models with Basic Model Interface (BMI).

In [16]:
sys.path.append("../src/")
import bmi_troute # This is the BMI t-route that we will be running from the file: bmi_troute.py 

In [17]:
model = bmi_troute.bmi_troute()

In [18]:
model.initialize(bmi_cfg_file='bmi_config_files/LowerColorado.yml')

creating supernetwork connections set
supernetwork connections set complete
setting waterbody and channel initial states ...
waterbody and channel initial states complete


In [19]:
model.set_value()

In [20]:
model.update()

In [21]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

,2021-08-24 06:05:00,2021-08-24 06:10:00,2021-08-24 06:15:00,2021-08-24 06:20:00,2021-08-24 06:25:00,2021-08-24 06:30:00,2021-08-24 06:35:00,2021-08-24 06:40:00,2021-08-24 06:45:00,2021-08-24 06:50:00,...,2021-08-24 11:15:00,2021-08-24 11:20:00,2021-08-24 11:25:00,2021-08-24 11:30:00,2021-08-24 11:35:00,2021-08-24 11:40:00,2021-08-24 11:45:00,2021-08-24 11:50:00,2021-08-24 11:55:00,2021-08-24 12:00:00
1611674,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,...,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063
1611676,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,...,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068
3763126,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,...,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257
3762962,0.002920,0.002920,0.002920,0.002920,0.002920,0.002920,0.002920,0.002920,0.002919,0.002919,...,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917
3762964,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,...,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942080041,0.431705,0.431712,0.431717,0.431722,0.431727,0.431731,0.431734,0.431737,0.431739,0.431742,...,0.431957,0.431962,0.431966,0.431970,0.431973,0.431976,0.431979,0.431981,0.431983,0.431985
942080042,0.303372,0.303373,0.303373,0.303373,0.303374,0.303374,0.303373,0.303373,0.303373,0.303373,...,0.303390,0.303390,0.303391,0.303392,0.303392,0.303393,0.303393,0.303393,0.303393,0.303393
942080043,0.763534,0.763538,0.763543,0.763547,0.763551,0.763556,0.763560,0.763565,0.763569,0.763573,...,0.763809,0.763814,0.763819,0.763824,0.763829,0.763834,0.763839,0.763844,0.763848,0.763852
942080044,3.873273,3.872474,3.871675,3.870877,3.870078,3.869282,3.868486,3.867692,3.866900,3.866110,...,3.824977,3.824215,3.823450,3.822685,3.821919,3.821154,3.820388,3.819622,3.818857,3.818094


In [22]:
model.get_current_time()

86400.0

Now utilize the update_until() function to run t-route for only a portion of the total time

In [23]:
model.initialize(bmi_cfg_file='bmi_config_files/LowerColorado.yml')

creating supernetwork connections set
supernetwork connections set complete
setting waterbody and channel initial states ...
waterbody and channel initial states complete


In [24]:
model.set_value()

In [25]:
model.update_until(21600) # input value is seconds. 21,600s = 6 hours

In [26]:
model.get_current_time()

21600.0

In [27]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

,2021-08-23 12:05:00,2021-08-23 12:10:00,2021-08-23 12:15:00,2021-08-23 12:20:00,2021-08-23 12:25:00,2021-08-23 12:30:00,2021-08-23 12:35:00,2021-08-23 12:40:00,2021-08-23 12:45:00,2021-08-23 12:50:00,...,2021-08-23 17:15:00,2021-08-23 17:20:00,2021-08-23 17:25:00,2021-08-23 17:30:00,2021-08-23 17:35:00,2021-08-23 17:40:00,2021-08-23 17:45:00,2021-08-23 17:50:00,2021-08-23 17:55:00,2021-08-23 18:00:00
1611674,0.000077,0.000077,0.000077,0.000077,0.000077,0.000077,0.000077,0.000077,0.000077,0.000076,...,0.000073,0.000073,0.000073,0.000072,0.000072,0.000072,0.000072,0.000072,0.000072,0.000072
1611676,0.000082,0.000082,0.000082,0.000082,0.000082,0.000082,0.000082,0.000082,0.000082,0.000082,...,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079
3763126,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,...,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257
3762962,0.002929,0.002929,0.002928,0.002928,0.002928,0.002928,0.002928,0.002928,0.002928,0.002928,...,0.002926,0.002926,0.002926,0.002926,0.002926,0.002926,0.002926,0.002926,0.002926,0.002926
3762964,0.001314,0.001314,0.001314,0.001314,0.001314,0.001314,0.001314,0.001314,0.001314,0.001314,...,0.001313,0.001313,0.001313,0.001313,0.001313,0.001313,0.001313,0.001313,0.001313,0.001313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942080041,0.430871,0.430877,0.430882,0.430887,0.430891,0.430895,0.430898,0.430901,0.430903,0.430905,...,0.431109,0.431114,0.431118,0.431122,0.431125,0.431128,0.431130,0.431132,0.431134,0.431136
942080042,0.303737,0.303734,0.303731,0.303728,0.303724,0.303720,0.303716,0.303712,0.303708,0.303704,...,0.303544,0.303542,0.303540,0.303537,0.303535,0.303533,0.303530,0.303528,0.303525,0.303522
942080043,0.763042,0.763043,0.763044,0.763044,0.763045,0.763046,0.763047,0.763047,0.763048,0.763048,...,0.763090,0.763092,0.763093,0.763095,0.763097,0.763099,0.763101,0.763102,0.763103,0.763104
942080044,3.947985,3.948170,3.948354,3.948538,3.948720,3.948903,3.949084,3.949264,3.949443,3.949621,...,3.953932,3.953872,3.953805,3.953732,3.953652,3.953567,3.953477,3.953382,3.953282,3.953178


Now call update() to run t-route for the remainder of the full model run

In [28]:
model.update()

In [29]:
model.get_current_time()

86400.0

In [30]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

,2021-08-24 06:05:00,2021-08-24 06:10:00,2021-08-24 06:15:00,2021-08-24 06:20:00,2021-08-24 06:25:00,2021-08-24 06:30:00,2021-08-24 06:35:00,2021-08-24 06:40:00,2021-08-24 06:45:00,2021-08-24 06:50:00,...,2021-08-24 11:15:00,2021-08-24 11:20:00,2021-08-24 11:25:00,2021-08-24 11:30:00,2021-08-24 11:35:00,2021-08-24 11:40:00,2021-08-24 11:45:00,2021-08-24 11:50:00,2021-08-24 11:55:00,2021-08-24 12:00:00
1611674,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,0.000065,...,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063
1611676,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,0.000071,...,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068,0.000068
3763126,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,...,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257,0.034257
3762962,0.002920,0.002920,0.002920,0.002920,0.002920,0.002920,0.002920,0.002920,0.002919,0.002919,...,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917,0.002917
3762964,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,0.001311,...,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310,0.001310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942080041,0.431705,0.431712,0.431717,0.431722,0.431727,0.431731,0.431734,0.431737,0.431739,0.431742,...,0.431957,0.431962,0.431966,0.431970,0.431973,0.431976,0.431979,0.431981,0.431983,0.431985
942080042,0.303372,0.303373,0.303373,0.303373,0.303374,0.303374,0.303373,0.303373,0.303373,0.303373,...,0.303390,0.303390,0.303391,0.303392,0.303392,0.303393,0.303393,0.303393,0.303393,0.303393
942080043,0.763534,0.763538,0.763543,0.763547,0.763551,0.763556,0.763560,0.763565,0.763569,0.763573,...,0.763809,0.763814,0.763819,0.763824,0.763829,0.763834,0.763839,0.763844,0.763848,0.763852
942080044,3.873273,3.872474,3.871675,3.870877,3.870078,3.869282,3.868486,3.867692,3.866900,3.866110,...,3.824977,3.824215,3.823450,3.822685,3.821919,3.821154,3.820388,3.819622,3.818857,3.818094
